### Views: keys, values and items

Views are special objects that support set behavior and also support iteration over the keys, values, and key/value pairs (items) in a dictionary.

A quick look at some common set operations:

In [1]:
s1 = {1, 2, 3}
s2 = {2, 3, 4}

Unions:

In [2]:
s1 | s2

{1, 2, 3, 4}

Intersections:

In [3]:
s1 & s2

{2, 3}

Differences:

In [4]:
s1 - s2

{1}

In [5]:
s2 - s1

{4}

Now let's look at these views:

In [6]:
d1 = {'a': 1, 'b': 2, 'c': 3}
d2 = {'c': 30, 'd': 4, 'e': 5}

We can iterate over the keys of a dictionary using the dictionary's iterator directly, or via the `keys` view:

In [7]:
for key in d1:
    print(key)

a
b
c


In [8]:
for key in d1.keys():
    print(key)

a
b
c


We can iterate over just the values of the dictionary:

In [9]:
for value in d1.values():
    print(value)

1
2
3


and over the items, as tuples, of the dictionary:

In [10]:
for item in d1.items():
    print(item)

('a', 1)
('b', 2)
('c', 3)


We can also unpack the tuples directly while iterating:

In [11]:
for k, v in d1.items():
    print(k, v)

a 1
b 2
c 3


These views are iterables, not just iterators:

In [13]:
keys = d1.keys()

In [14]:
list(keys)

['a', 'b', 'c']

In [15]:
list(keys)

['a', 'b', 'c']

As you can see we can iterate over and over on the same view.

The order in which keys, value and items are returned during iteration match - as long as the dictionary has not changed in-between.

So for example, the following expression will always evaluate to true:

In [16]:
list(d1.items()) == list(zip(d1.keys(), d1.values()))

True

Views are dynamic, in the sense that if something changes in the dictionary, the views immediately reflect the change - that's because the views do not themselves contain data, they simply have extra bits of functionality that uses the dictionary as the source of truth.

In [17]:
keys

dict_keys(['a', 'b', 'c'])

In [18]:
d1['z'] = 10

In [19]:
keys

dict_keys(['a', 'b', 'c', 'z'])

In [20]:
del d1['z']

In [21]:
keys

dict_keys(['a', 'b', 'c'])

Now, the interesting thing is that some of these views also exhibit set behaviors.

In [22]:
print(d1)
print(d2)

{'a': 1, 'b': 2, 'c': 3}
{'c': 30, 'd': 4, 'e': 5}


We can find all the keys that are in both `d1` and `d2`:

In [23]:
print(type(d1.keys()), d1.keys())
print(type(d2.keys()), d2.keys())
union = d1.keys() | d2.keys()
print(type(union), union)

<class 'dict_keys'> dict_keys(['a', 'b', 'c'])
<class 'dict_keys'> dict_keys(['c', 'd', 'e'])
<class 'set'> {'e', 'a', 'c', 'd', 'b'}


One thing to really watch out for here: once we start performing set like operations, the result is a true `set`, and although ordering in the views is guaranteed, ordering in the resulting sets are **not** as you can see from the example above!

We can also find the keys that are in both `d1` and `d2`:

In [24]:
d1.keys() & d2.keys()

{'c'}

We can also find the keys that are only in `d1` but not in `d2`:

In [25]:
d1.keys() - d2.keys()

{'a', 'b'}

The same works with items as well:

In [26]:
d1.items() | d2.items()

{('a', 1), ('b', 2), ('c', 3), ('c', 30), ('d', 4), ('e', 5)}

You'll notice that `('c', 3)` and `('c', 30)` are distinct elements, hence they show up as individual elements in the result.

Values on the other hand are more problematic. Keys in a dictionary must be hashable, and set elements must also be hashable, so it's not a problem creating a set of keys for example. But what about values? These need not be unique or hashable. And items for that matter? The first element of the tuple must be hashable since it's the key, but the value?

In [27]:
d3 = {'a': [1, 2], 'b': [3, 4]}
d4 = {'b': [30, 40], 'c': [5, 6]}

In [28]:
d3.values()

dict_values([[1, 2], [3, 4]])

Can we perform some set operations on the values?

In [29]:
d3.values() | d4.values()

TypeError: unsupported operand type(s) for |: 'dict_values' and 'dict_values'

The answer is no, the `values` view does not behave like a set - it can't because there is no guarantee the values are unique and hashable.

What's interesting though is that `items` does have unique values (since the keys are unique), but the values may or may not be hashable as in the example of `d3` and `d4`:

In [30]:
print(d3)
print(d4)

{'a': [1, 2], 'b': [3, 4]}
{'b': [30, 40], 'c': [5, 6]}


In [31]:
d3.items() | d4.items()

TypeError: unhashable type: 'list'

As you can see, `items`, in this case also does not exhibit set like capabilities.

But that's not always the case. Let's go back to our first example:

In [32]:
print(d1)
print(d2)

{'a': 1, 'b': 2, 'c': 3}
{'c': 30, 'd': 4, 'e': 5}


In [33]:
d1.items() | d2.items()

{('a', 1), ('b', 2), ('c', 3), ('c', 30), ('d', 4), ('e', 5)}

Aha! In this case `items` **does** behave like a set - that's because the values are all hashable!

#### Example 1

Let's take a look at a practical example of using these views for something other than plain iteration:

Let's say we have two dictionaries, and we want to create a new dictionary that contains all the items whose keys are in both dictionaries.
We want the value in the new dictionary to be a tuple containing all the values from both dictionaries:

In [34]:
d1 = {'a': 1, 'b': 2, 'c': 3}
d2 = {'b': 2, 'c': 30, 'd': 4}

In [35]:
k1 = d1.keys()
k2 = d2.keys()
k1 & k2

{'b', 'c'}

So we have now identified the common keys, all that's left to do is build a dictionary from those keys and the corresponding values.

We can use a simple loop to do this:

In [36]:
new_dict = {}
for key in d1.keys() & d2.keys():
    new_dict[key] = (d1[key], d2[key])
print(new_dict)

{'c': (3, 30), 'b': (2, 2)}


But, a dictionary comprehension would be a better approach here:

In [37]:
new_dict = {key: (d1[key], d2[key]) for key in d1.keys() & d2.keys()}
print(new_dict)

{'c': (3, 30), 'b': (2, 2)}


#### Example 2

Let's tweak this a bit and generate a new dictionary, again containing just the common keys, but whose value is either the common value, or if the underlying dictionaries have different values for the same key, choose the values from the second dictionary, discarding the values from the first.

The approach is going to be almost identical to the previous example.

Let's just see which value we want to use for both cases (same values, different values):
* same values: pick value from `d1` or `d2` (since values are the same it does not matter)
* different values: pick value from `d2`

As you can see, in both cases we just need to pick the value in `d2`.

In [38]:
d1 = {'a': 1, 'b': 2, 'c': 3}
d2 = {'b': 2, 'c': 30, 'd': 4}
new_dict = {key: d2[key] for key in d1.keys() & d2.keys()}
print(new_dict)

{'c': 30, 'b': 2}
